<a href="https://colab.research.google.com/github/AG-X09/Defensive-Approximation/blob/main/Defensive_approximation_ImageNet_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [49]:
cd /content/gdrive/MyDrive/

/content/gdrive/MyDrive


# Approximate BFloat16 multiplier

In [1]:
#Approximate 4x4 array multiplier (using the approximate mirror adder AMA5)
def appx_multiplier4x4_AMA5(A,B):

    S = 0
    if (A == 0) or (B == 0):
        S = 0
    elif (A == 1):
        S = B
    elif (A % 2 == 0) & (A>1):
        if (B < 8): 
            S = 0
        else:
            S = 32 * (A/2)
    else:
        if (B < 8): 
            S = B
        else:
            S = B + 32 * (A-1)/2
    return S  

In [2]:
#Approximate 8x8 array multiplier
def appx_multiplier8x8(a,b):

    a0b0 = appx_multiplier4x4_AMA5(int(a[4:8],2), int(b[4:8],2))
    a1b0 = appx_multiplier4x4_AMA5(int(a[0:4],2), int(b[4:8],2))
    a0b1 = appx_multiplier4x4_AMA5(int(a[4:8],2), int(b[0:4],2))
    a1b1 = appx_multiplier4x4_AMA5(int(a[0:4],2), int(b[0:4],2))
    S = (a0b0 + (a1b0 + a0b1)*16 + a1b1*256) 
    S = format(int(S), '016b')
    return S

In [3]:
#Approximate BFloat16 multiplier
def BF_appx_mul(A,B):

  if (abs(A)<1e-36) or (abs(B)<1e-36) or (A == 0) or (B==0):
    s = 0
  else:
    S = ['0','00000000','00000000000000000000000']

    a = ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack('!f', A))  
    b = ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack('!f', B)) 

    sign_ab = int(a[0])^int(b[0])

    exponent_ab = int(a[1:9],2) + int(b[1:9],2) - 127
    if exponent_ab<1:
        s = 0
    else:
        mantissa_ab = appx_multiplier8x8('1'+a[9:16],'1'+b[9:16])
  
        if mantissa_ab[0] == '1':
            final_mantissa = mantissa_ab[1:8]
            exponent_ab = exponent_ab + 1
        else:
            final_mantissa = mantissa_ab[2:9] 
        S = ["{}".format(sign_ab), format(exponent_ab,'08b'), final_mantissa +'0000000000000000']
        S = ''.join(S) 
        s = struct.unpack('!f',struct.pack('!I', int(S, 2)))[0]

  return s 

In [4]:
from torchvision import models, datasets, transforms
import torch
import numpy as np
import os

VAL_SIZE = 50000


device = torch.device( "cpu")

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from joblib import Parallel, delayed

# Approximate Convolution Layer

In [5]:
class convAppx(torch.autograd.Function):

    @staticmethod
    def forward(ctx, X, weight, bias, padding, stride):

        ctx.save_for_backward(X, weight, bias)        
        (m, n_C_prev, n_H_prev, n_W_prev) = X.shape
        (n_C, n_C_prev, f, f) = weight.shape

        n_H = ((n_H_prev - f + 2 * padding[0]) // stride[0]) + 1
        n_W = ((n_W_prev - f + 2 * padding[0]) // stride[0]) + 1



        def appx_mul(A,B):
            window = np.zeros((A.shape))
            for l in range(A.shape[0]):
              for j in range(A.shape[1]):
                for k in range(A.shape[2]):
                  window[l,j,k] = BF_appx_mul(A[l,j,k],B[l,j,k])  #A[l,j,k]*B[l,j,k]
            return np.sum(window)

        def mul_channel( weight,bias, x_pad, n_H, n_W,f):
              Z = np.zeros(( n_H, n_W ))
              for h in range(n_H):
                  for w in range(n_W):
                      vert_start = h
                      vert_end = vert_start + f
                      horiz_start = w
                      horiz_end = horiz_start + f
            
                      x_slice = x_pad[:, vert_start:vert_end, horiz_start:horiz_end]  
                      Z[ h, w] = appx_mul(x_slice,weight)  #torch.matmul(A,B)
                      Z[ h, w] += bias
              return Z
      
        X_pad = F.pad(X, (padding[0],padding[0],padding[0],padding[0]))
        weight = weight.data.numpy()
        bias = bias.data.numpy()
        X_pad = X_pad.data.numpy()

        Z = np.zeros((m, n_C, n_H, n_W ))     
        import os
        from joblib import dump, load
        
        folder = './joblib_memmap'
        try:
            os.mkdir(folder)
        except FileExistsError:
            pass
        data_ = X_pad[0]
        
        data_filename_memmap = os.path.join(folder, 'data___memmap')
        dump(data_, data_filename_memmap)
        data_ = load(data_filename_memmap, mmap_mode='r')         

    
        for i in range(m):
            for c in range(n_C):
                Z[i,c] = mul_channel( weight[c, :, :, :],bias[c], data_, n_H, n_W, f)       
            #To use Joblib Parallel uncomment this          
            #x_pad = X[0]
            #Z[0] = Parallel(n_jobs=8)(delayed(mul_channel)( weight[c, :, :, :],bias[c], X_pad[0], n_H, n_W, f)  for c in  range(n_C) )  
        return torch.from_numpy(Z).float() 

    @staticmethod
    def backward(ctx, grad_output):
        x, weight, bias = ctx.saved_tensors 

        grad_input = grad_weight = grad_bias = None


        def convolutionBackward(dconv_prev, conv_in, weight, padding =1, stride=1):
            (m, n_C_prev, n_H_prev, n_W_prev) = conv_in.shape
            (n_C, n_C_prev, f, f) = weight.shape
            (m, n_C, n_H, n_W) = dconv_prev.shape

            dA_prev = torch.zeros((m, n_C_prev, n_H_prev, n_W_prev))
            dW = torch.zeros((n_C, n_C_prev, f, f))
            db = torch.zeros((n_C))
            X_pad = F.pad(conv_in, (padding,padding,padding,padding))
            dA_prev_pad = F.pad(dA_prev, (padding,padding,padding,padding))

            for i in range(m):
                x_pad = X_pad[i]
                da_prev_pad = dA_prev_pad[i]
              
                for c in range(n_C):
                    for h in range(n_H):
                        for w in range(n_W):
                            vert_start = h + h * (stride - 1)
                            vert_end = vert_start + f
                            horiz_start = w + w * (stride - 1)
                            horiz_end = horiz_start + f

                            x_slice = x_pad[:, vert_start:vert_end, horiz_start:horiz_end]

                            da_prev_pad[:, vert_start:vert_end, horiz_start:horiz_end] += weight[c, :, :, :] * dconv_prev[i, c, h, w]
                        
                            dW[c,:,:,:] += x_slice * dconv_prev[i, c, h, w]
                            
                            db[c] += dconv_prev[i, c, h, w]  
                if padding == 0:
                  dA_prev[i, :, :, :] = da_prev_pad[:]
                else:
                  dA_prev[i, :, :, :] = da_prev_pad[:, padding:-padding, padding:-padding] 
            return dA_prev, dW, db
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
        grad_input, grad_weight, grad_bias = convolutionBackward(grad_output, x, weight)
        grad_bias = grad_bias.squeeze()
        return grad_input, grad_weight, grad_bias, None,None   

class MyConv2d(nn.Module):
    def __init__(self, n_channels, out_channels, kernel_size , padding, stride,dilation=1):
        super(MyConv2d, self).__init__()

        self.kernel_size = (kernel_size, kernel_size)
        self.kernal_size_number = kernel_size * kernel_size
        self.out_channels = out_channels
        self.dilation = (dilation, dilation)
        self.padding = (padding, padding)
        self.stride = (stride, stride)
        self.n_channels = n_channels
        self.weight = nn.Parameter(torch.rand(self.out_channels, self.n_channels, self.kernel_size[0] , self.kernel_size[1] ))
        self.bias = nn.Parameter(torch.rand(self.out_channels))

    def forward(self, x):
        res = convAppx.apply(x, self.weight, self.bias, self.padding, self.stride)

        return res

Approximate Fully Connected Layer

In [30]:
class linear_appx(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, weight, bias):
        ctx.save_for_backward(input, weight, bias)

        input = input.data.numpy()
        weight = weight.data.numpy()
        bias = bias.data.numpy()
        def appx_mul(A,B):
          window = np.zeros((A.shape[0],B.shape[1] ))
          for k in range(A.shape[0]):
            for l in range(B.shape[1]):
              for j in range(A.shape[1]):
                  window[k,l] += BF_appx_mul(A[k,j],B[j,l])  #A[k,j]*B[j,l]  #
          return window

        output = appx_mul(input,np.transpose(weight)) + bias
        #print(output)
        return torch.from_numpy(output).float()

    @staticmethod
    def backward(ctx, grad_output):
        input, weight, bias = ctx.saved_tensors

        grad_input = grad_output.mm(weight.float())
        grad_weight = grad_output.t().mm(input.float())
        grad_bias = grad_output.sum(0)
        return grad_input, grad_weight, grad_bias


class MyLinear(nn.Module):
    def __init__(self,in_features, out_features ):
        super(MyLinear, self).__init__()
        self.fn = linear_appx.apply
        self.in_features = in_features
        self.out_features = out_features
        self.weight = torch.nn.Parameter(torch.randn(out_features, in_features))
        self.bias = torch.nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        x = self.fn(x, self.weight, self.bias)
        return x

In [31]:

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url
from typing import Union, List, Dict, Any, cast


__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn_appx', 'vgg19_appx',
]


model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}


# Define Exact VGG Model 

In [32]:

class VGG(nn.Module):

    def __init__(
        self,
        features: nn.Module,
        num_classes: int = 1000,
        init_weights: bool = False
    ) -> None:
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers_exact(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

def _vgg_(arch: str, cfg: str, batch_norm: bool, pretrained: bool, progress: bool, **kwargs: Any) -> VGG:
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers_exact(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        #state_dict = torch.load('vgg11-bbd30ac9.pth')
        #model.load_state_dict(state_dict)
        model.load_state_dict(model_zoo.load_url(model_urls['vgg11']))
    return model


def vgg11(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 11-layer model (configuration "A") from
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg_('vgg11', 'A', False, pretrained, progress, **kwargs)


def vgg11_bn(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 11-layer model (configuration "A") with batch normalization
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg_('vgg11_bn', 'A', True, pretrained, progress, **kwargs)


# Define Approximate VGG Model 

In [33]:
import torch.utils.model_zoo as model_zoo
class VGG_appx(nn.Module):

    def __init__(
        self,
        features: nn.Module,
        num_classes: int = 1000,
        init_weights: bool = True
    ) -> None:
        super(VGG_appx, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            MyLinear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            MyLinear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            MyLinear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        print("done!")
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = MyConv2d(in_channels, v, 3,1,1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs: Dict[str, List[Union[str, int]]] = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def _vgg(arch: str, cfg: str, batch_norm: bool, pretrained: bool, progress: bool, **kwargs: Any) -> VGG_appx:
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG_appx(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        #state_dict = torch.load('vgg11-bbd30ac9.pth')
        #model.load_state_dict(state_dict)
        model.load_state_dict(model_zoo.load_url(model_urls['vgg11']))
    return model



def vgg11_appx(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG_appx:
    r"""VGG 19-layer model (configuration "E")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg11', 'A', False, pretrained, progress, **kwargs)


def vgg11_bn_appx(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG_appx:
    r"""VGG 19-layer model (configuration 'E') with batch normalization
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg11_bn', 'A', True, pretrained, progress, **kwargs)


In [62]:
#PATH = 'vgg11-bbd30ac9.pth' 
model_name = "vgg11"
model = vgg11(pretrained= True, progress = True)

model.load_state_dict(model_zoo.load_url(model_urls['vgg11']))
model.eval()
# Send the model to GPU/CPU
model = model.to(device)


model_name = "vgg11_appx"
model_appx = vgg11_appx(pretrained= True, progress = True)

model_appx.load_state_dict(model_zoo.load_url(model_urls['vgg11']))
model_appx.eval()
# Send the model to GPU/CPU
model_appx = model_appx.to(device)

In [ ]:
# Create shaffeled 1000 sample
batch_size = 1000
dataloaders = {}
#for img_size in [224, 299]:

img_size = 224
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
val_dataset = datasets.ImageFolder(root=os.path.join('./val/'), transform=val_transform)  #Check "ImageNet_validation_data_extraction.ipynb" to download and extract ImageNet validation data

dataloaders[img_size] = torch.utils.data.DataLoader(
    val_dataset, 
    batch_size=batch_size,                                             
    shuffle=False, 
    num_workers= max(1, multiprocessing.cpu_count() - 2),
)
        
loader = dataloaders[img_size]
with torch.set_grad_enabled(False):
    for i, (x_val, y_val) in enumerate(loader):
      x_val, y_val = x_val, y_val
      break

In [ ]:
i = 228
x = x_val[i].unsqueeze(0)
y = y_val[i].unsqueeze(0) - 1

out_exact = torch.nn.functional.softmax(model(x), dim=1)
print("Prediction exact:",out_exact.argmax(1))

out_appx = torch.nn.functional.softmax(model_appx(x), dim=1)
print("Prediction appx:",out_appx.argmax(1))